# 텐서플로우 MNIST 리프트 앤 시프트 연습문제

이 연습용 노트북의 경우, SageMaker 스튜디오에서 `Python 3 (TensorFlow 2.3 Python 3.7 CPU Optimized)` 커널을 사용하거나 클래식 SageMaker 노트북 인스턴스에서 `conda_tensorflow2_p37`을 사용할 수 있어야 합니다.


---

## 소개

데이터사이언스 팀의 새 동료(SageMaker에 익숙하지 않은)가 Keras로 이미지 분류 문제를 해결하기 위해 멋진 노트북을 작성했습니다: [Local Notebook.ipynb](Local%20Notebook.ipynb).

이전에 작업하던 간단한 MNIST 데이터 세트에서는 잘 작동했지만, 이제는 더 크고 어려운 문제를 해결하기 위해 SageMaker의 몇 가지 기능을 활용하고 싶어합니다.

**로컬 노트북 코드를 활용하여 SageMaker를 효과적으로 사용하는 방법을 알아봅시다**



## 시작하기

먼저, **[Local Notebook.ipynb](Local%20Notebook.ipynb) 노트북을 실행**해 어떤 단계가 필요한지 검토하세요.

**이 노트북**에는 코드를 마이그레이션하는 데 사용할 수 있는 구조가 나와 있고, 높은 수준에서 변경해야 할 몇 가지 사항이 나열되어 있습니다. 여기서 직접 작업하거나 이 노트북을 복제하여 원본의 변경되지 않은 사본을 보관할 수 있습니다.

먼저 MVP 목표를 염두에 두고 섹션을 진행하세요(SageMaker 학습 작업을 통해 S3의 데이터에 모델 맞추기, SageMaker 엔드포인트를 통해 모델 배포/사용하기). 마지막에는 더 고급 기능을 도입하기 위한 확장 연습이 있습니다.


## 종속성

모든 임포트를 처음에 나열하면 스크립트/파일을 실행하기 위한 요구 사항을 미리 투명하게 유지하는 데 도움이 되며, Python의 공식 [PEP 8](https://www.python.org/dev/peps/pep-0008/#imports)을 포함한 거의 모든 스타일 가이드에 명시되어 있습니다.


In [ ]:
!pip install "ipycanvas<0.13" "ipywidgets<8" matplotlib

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import glob
import os

# External Dependencies:
import matplotlib.pyplot as plt
import numpy as np

# TODO: What else will you need?
# Have a look at the documentation: https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html
# to see which libraries need to be imported to use sagemaker and the tensorflow estimator

## 데이터 준비

AWS의 오픈 데이터 리포지토리에서 이미지 데이터를 다운로드하고 [Local Notebook.ipynb](Local%20Notebook.ipynb)에서 했던 것처럼 하위 집합을 샘플링해 봅시다.

이 노트북에서 어떤 데이터를 업로드할지, 그리고 S3에서 어디에 저장할지 **이해했는지 확인**한 다음, 나머지 작업을 하는 동안 업로드 실행을 시작하세요.

In [ ]:
local_dir = "/tmp/mnist"
training_dir = f"{local_dir}/training"
testing_dir = f"{local_dir}/testing"

# Download the MNIST data from the Registry of Open Data on AWS
!rm -rf {local_dir}
!mkdir -p {local_dir}
!aws s3 cp s3://fast-ai-imageclas/mnist_png.tgz {local_dir} --no-sign-request

# Un-tar the MNIST data, stripping the leading path element; this will leave us with directories
# {local_dir}/testing/ and {local_dir/training/
!tar zxf {local_dir}/mnist_png.tgz -C {local_dir}/ --strip-components=1 --no-same-owner

# Get the list of files in tne training and testing directories recursively
train_files = sorted(list(glob.iglob(os.path.join(training_dir, "*/*.png"), recursive=True)))
test_files = sorted(list(glob.iglob(os.path.join(testing_dir, "*/*.png"), recursive=True)))

print(f"Training files: {len(train_files)}")
print(f"Testing files:  {len(test_files)}")

# Reduce the data by keeping every Nth file and dropping the rest of the files.
reduction_factor = 2
train_files_to_keep = train_files[::reduction_factor]
test_files_to_keep = test_files[::reduction_factor]

print(f"Training files kept: {len(train_files_to_keep)}")
print(f"Testing files kept:  {len(test_files_to_keep)}")

# Delete all the files not to be kept
for fname in set(train_files) ^ set(train_files_to_keep):
    os.remove(fname)

for fname in set(test_files) ^ set(test_files_to_keep):
    os.remove(fname)

print("Done!")

## 실행 역할, 세션 및 S3 버킷 설정하기

이제 로컬 디렉터리에서 데이터를 다운로드하고 축소했으므로 Amazon S3에 업로드하여 Amazon Sagemaker 학습에 사용할 수 있도록 해야 합니다. 

지정하는 것부터 시작하겠습니다:
- 학습 및 모델 데이터에 사용할 S3 버킷과 접두사를 지정합니다. 이 버킷은 노트북 인스턴스, 학습 및 호스팅과 동일한 리전 내에 있어야 합니다. 버킷을 지정하지 않으면 SageMaker SDK는 동일한 리전에서 미리 정의된 명명 규칙에 따라 기본 버킷을 생성합니다. 
- SageMaker에 데이터 액세스 권한을 부여하는 데 사용되는 IAM Role(역할) ARN입니다. SageMaker 파이썬 SDK에서 **get_execution_role** 메서드를 사용하여 가져올 수 있습니다.

In [ ]:
# TODO: This is where you can setup execution role, session and S3 bucket.
# 1. Setup the SageMaker role
role = ?
# 2. Setup the SageMaker session
sess = ?
# 3. Setup the SageMaker default bucket
bucket_name = ?

# Have a look at the previous examples to find out how to do it

## Amazon S3에 데이터 업로드
다음은 SageMaker 학습을 위해 이미지를 Amazon S3에 업로드하는 부분입니다. 이전 예제에서 [aws s3 sync](https://docs.aws.amazon.com/cli/latest/reference/s3/sync.html) CLI 명령어를 사용하여 업로드하는 방법을 참고할 수 있습니다. 상위 명령어인 ```aws s3 sync``` 명령은 대상 버킷과 소스 디렉터리의 콘텐츠를 동기화합니다. 소스에 없는 오브젝트를 대상에서 제거할 수 있는 ```--delete``` 옵션과 파일이나 오브젝트를 제외하거나 제외하지 않도록 필터링하는 ```--exclude``` 또는 ```--include``` 옵션 등을 사용할 수 있습니다.

> ⏰ **확인:** Amazon S3에 업로드하는 데는 일반적으로 `reduction_factor`을 2로 가정할 때 약 2~3분이 소요됩니다.

In [ ]:
# TODO: This is where you upload the training images using `aws s3 sync`.
# Fill in the missing source local directory and the target S3 bucket and folder in the command below.
!aws s3 sync --quiet --delete ??? ??? --exclude "*.tgz" && echo "Done!"

[Amazon S3 콘솔](https://s3.console.aws.amazon.com/s3/home)에서 버킷을 찾아 데이터가 업로드되었는지 확인할 수 있습니다. 이미지 폴더가 예상대로 표시되나요?

## 데이터 입력("채널") 구성

코드 초안에는 **2개의 데이터 세트**가 있습니다: 하나는 학습용, 다른 하나는 테스트/검증용입니다. (분류를 위해서는 각 이미지의 폴더 위치가 레이블로 충분합니다.)

SageMaker 용어로 각 입력 데이터 세트는 "채널"이며, 원하는 대로 이름을 지정할 수 있습니다. 다만 각각의 이름을 일관성 있게 부르는 것이 중요합니다!

간단한 입력 구성의 경우 채널 사양은 폴더의 S3 URI일 수 있습니다. 고급 옵션을 구성하려면 SageMaker SDK에 [s3_input](https://sagemaker.readthedocs.io/en/stable/inputs.html) 클래스가 있습니다.


In [ ]:
# TODO: Define your 2 data channels
# The data can be found in: "s3://{bucket_name}/mnist/training" and "s3://{bucket_name}/mnist/testing"

inputs = # Look at the previous example to see how the inputs were defined

## 알고리즘(Estimator(추정기)) 구성 및 실행

여기 노트북에서 이 데이터를 로드하고 맞추는 대신, 필요에 따라 크기를 조정할 수 있는 별도의 컨테이너에서 코드를 실행하기 위해 SageMaker SDK를 통해 [TensorFlow Estimator](https://sagemaker.readthedocs.io/en/stable/sagemaker.tensorflow.html#tensorflow-estimator)를 만들겠습니다.

["Using TensorFlow with the SageMaker Python SDK"](https://sagemaker.readthedocs.io/en/stable/using_tf.html#train-a-model-with-tensorflow) 문서에 이 과정에 대한 좋은 개요가 나와 있습니다. 견적기를 **스크립트 모드**(기존 기본 레거시 모드보다 따라하기 쉬움)와 **파이썬 3**로 실행해야 합니다.

**코드를 포팅할 진입점으로**[src/main.py](src/main.py) 파일을 사용하세요. 이 파일은 몇 가지 기본 힌트와 함께 이미 생성되어 있습니다.


In [ ]:
# TODO: Create your TensorFlow estimator

# Note the TensorFlow class inherits from some cross-framework base classes with additional
# constructor options:
# https://sagemaker.readthedocs.io/en/stable/estimators.html
# https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html?highlight=%20TrainingInput#create-an-estimator

# We are using tensorflow 1.14 and python 3
# You can reuse the metrics definition from the previous example
# (Optional) Look at the Tensorflow script and try to pass new hyperparameters

estimator = ?

SageMaker TrainingJob에서 실제 학습을 실행하기 전에 아래 코드를 사용하여 로컬에서 먼저 실행해보는 것이 좋습니다. 오류가 있는 경우 SageMaker TrainingJob을 사용하여 실행하기 전에 먼저 오류를 수정할 수 있습니다.

In [ ]:
!python3 src/main.py --train {training_dir} --test {testing_dir} --output-data-dir data/local-output --model-dir data/local-model --epochs=1 --batch-size=128

SageMaker 트레이닝 작업에서 스크립트를 사용해 볼 준비가 되면 이전 연습에서 했던 것처럼 `estimator.fit()`을 호출할 수 있습니다:

In [ ]:
# TODO: Call estimator.fit

## 모델 배포 및 사용(실시간 추론)

학습 작업이 완료되고 모델을 올바른 TensorFlow Serving 호환 형식으로 저장했다면 이제 모델을 실시간 엔드포인트에 배포하는 것은 매우 간단합니다.

[Estimator API](https://sagemaker.readthedocs.io/en/stable/estimators.html)를 사용하면 이 작업을 수행할 수 있습니다.

In [ ]:
# TODO: Deploy a real-time endpoint

예제 노트북의 아키텍처를 검토하면서, **표준화된 0-1 픽셀 값**과 **단일 컬러 채널 차원**을 가진 **28x28** 이미지 텐서의 **배치**를 받아들이도록 모델을 설정했습니다.

스크립트에 [사용자 정의 추론 전처리](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/deploying_tensorflow_serving.html#providing-python-scripts-for-pre-post-processing)를 추가하지 않았다고 가정하면(예: 인코딩된 JPEG/PNG 또는 임의의 이미지 모양을 허용하기 위해), 엔드포인트를 사용할 때 동일한 형식을 복제해야 합니다.

## 추가 개선 사항

기본적인 학습/배포/호출 주기가 작동하고 있다면 축하드립니다! 노트북에서 실험하지만 확장 가능한 하드웨어에서 작업을 실행하는 이 핵심 패턴은 SageMaker 데이터사이언스 워크플로우의 핵심입니다.

하지만 도구를 더 잘 사용할 수 있는 방법은 아직 많이 남아 있습니다: 다음 도전 과제를 계속 읽어보세요!

### 1. SageMaker 관리형 스팟 모드로 훈련 비용을 쉽게 절감하세요.

AWS 스팟 인스턴스를 사용하면 표준 온디맨드 가격 대비 최대 90% 할인된 가격으로 AWS 클라우드에서 사용하지 않는 용량을 활용할 수 있습니다! 이와 같은 소규모 작업의 경우, 이 할인 혜택을 받으려면 Estimator 생성자에 몇 가지 매개 변수를 추가하는 것만큼이나 쉽습니다:

https://sagemaker.readthedocs.io/en/stable/estimators.html

일반적으로 스팟 용량은 순간적인 수요에 따라 중단될 수 있기 때문에 할인된 요금으로 제공된다는 점에 유의하세요. 장기간 실행되는 훈련 작업은 체크포인트 저장 및 로딩을 구현하여 도중에 중단되더라도 효율적으로 다시 시작할 수 있도록 해야 합니다. 자세한 내용은 [SageMaker Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/)의 [Managed Spot Training in Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/model-managed-spot-training.html) 페이지에서 확인할 수 있습니다.


### 2. 알고리즘 매개변수화

매번 `main.py` 스크립트를 수정하지 않고도 런타임에 알고리즘의 파라미터를 변경할 수 있으면 코드의 재사용성을 높이는 데 도움이 됩니다... 하지만 자동 하이퍼파라미터 튜닝을 위한 전제 조건이기 때문에 더욱 그렇습니다!

작업 매개변수 파싱은 이상적으로는 별도의 함수에 포함시켜야 하며, 모범 사례로 **둘 다** 명령줄 플래그를 통해 설정 값을 허용해야 합니다([official MXNet MNIST example](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/mxnet_mnist/mnist.py)) **와** [SageMaker Hyperparameter environment variable(s)](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-container-environmental-variables-user-scripts.html)에 나와 있는 것처럼). 공식 MXNet 예제에서 이미 채널에서와 마찬가지로 알고리즘 하이퍼파라미터에 환경 변수 기반 기본값을 설정하여 개선할 수 있을까요?

**에포크** 및 **배치 크기**를 선택적 파라미터로 허용하도록 작업을 리팩터링하고, [Estimator API](https://sagemaker.readthedocs.io/en/stable/estimators.html)를 통해 각 훈련 실행 전에 이를 설정하는 방법을 보여주세요.


### 3. 네트워크 하이퍼파라미터 조정

이전과 동일한 접근 방식을 다시 사용하여 네트워크 구조의 일부 기능을 파라미터화합니다: 아마도 `Conv2D` 커널의 크기일까요? 네트워크 내 레이어의 수, 유형, 노드 수 또는 활성화 기능? 샘플 아키텍처에서 너무 멀리 벗어날 필요가 없습니다!

매번 다른 하이퍼파라미터를 사용하여 `estimator.fit()`을 수동으로(또는 프로그래밍 방식으로) 호출하는 대신 SageMaker의 Bayesian Hyperparameter Tuning 기능을 사용하여 공간을 보다 효율적으로 탐색할 수 있습니다!

SageMaker SDK 문서에는 HyperparameterTuner 사용에 대한 훌륭한 [overview](https://sagemaker.readthedocs.io/en/stable/overview.html#sagemaker-automatic-model-tuning)가 있으니, 막히는 부분이 있으면 참고하세요.

먼저, 최적화할 특정 **메트릭**을 정의해야 하는데, 이는 알고리즘의 콘솔 로그에서 메트릭 값을 스크랩하는 방법에 대한 스펙입니다. 

그런 다음, [\*Parameter](https://sagemaker.readthedocs.io/en/stable/tuner.html) 클래스(`ContinuousParameter`, `IntegerParameter` 및 `CategoricalParameter`)를 사용하여 조합을 최적화하려는 하이퍼파라미터에 대한 적절한 범위를 정의합니다.

원본 estimator, 목표 지표 및 파라미터 범위를 정의한 후에는 [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/tuner.html)를 생성하고 이를 사용하여 단일 모델 훈련 작업 대신 하이퍼파라미터 튜닝 작업을 시작할 수 있습니다.

하이퍼파라미터 튜닝 실행의 최대 총 훈련 작업 수와 최대 병렬 작업 수를 선택할 때 실행 시간과 리소스 소비 가능성에 주의하세요... SageMaker 콘솔을 통해 진행 중인 하이퍼파라미터 튜닝 작업을 언제든지 확인하고 취소할 수 있습니다.


### 추가 도전 과제

시간이 있다면, 다음 도전 과제는 더 까다롭지만 SageMaker 지식을 더욱 확장할 수 있습니다!

**Batch Transform / Additional Inference Formats**: 이 노트북에서 설명한 것처럼 배포된 엔드포인트는 요청에 대해 특정 텐서 데이터 형식을 기대합니다. 이는 일괄 추론을 위해 동일한 모델의 용도를 변경하는 통상적으로 간단한 작업을 복잡하게 만듭니다(S3의 데이터는 JPEG 형식이므로). SageMaker 텐서플로우 SDK 문서는 ["Create Python Scripts for Custom Input and Output Formats"](https://sagemaker.readthedocs.io/en/stable/using_tf.html#create-python-scripts-for-custom-input-and-output-formats) 섹션에서 사용자 지정 형식을 허용하는 방법에 대한 지침을 제공합니다. 실시간 엔드포인트로 배포할 때 JPEG 요청을 수락하도록 알고리즘을 리팩터링할 수 있다면, 간단한 `estimator.transformer()` call로 S3의 이미지에 대해 일괄적으로 [Transformer](https://sagemaker.readthedocs.io/en/stable/transformer.html)로 실행할 수 있습니다.

**최적화된 훈련 형식**: 이와 같은 데이터 세트(작은 객체가 많이 포함된)는 Keras가 배포하는 표준 Numpy 형식으로 변환하거나(단 4개의 파일 X_train, Y_train, X_test, Y_test), 데이터를 미리 다운로드하는 대신 [SageMaker Pipe Mode](https://aws.amazon.com/blogs/machine-learning/using-pipe-input-mode-for-amazon-sagemaker-algorithms/)로 *스트리밍*하면 알고리즘에 로드하는 데 훨씬 적은 시간이 소요될 수 있습니다.

**실험 추적**: [SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) 기능을 사용하면 여러 관련 실험(예: 서로 다른 HPO 실행 또는 HPO와 일반 모델 훈련 작업 간)에서 실험을 보다 구조적으로 추적할 수 있습니다. 이 노트북에서 실험을 추적하는 방법에 대한 지침은 [official SageMaker Experiments Example](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/sagemaker-experiments)를 사용할 수 있으며, [SageMaker Experiments SDK Docs](https://sagemaker-experiments.readthedocs.io/en/latest/)는 다른 Python 모듈이기 때문에 별도로 유지 관리된다는 점에 유의해야 합니다.




## 정리

비용을 절약하려면 더 이상 필요하지 않은 영구 리소스를 정리해야 합니다: 이 중 가장 중요한 것은 실시간 예측 엔드포인트와 이 SageMaker 노트북 인스턴스입니다.

SageMaker SDK [Predictor](https://sagemaker.readthedocs.io/en/stable/predictors.html) 클래스는 실시간 예측 엔드포인트를 정리할 수 있는 인터페이스를 제공하며, 작업이 끝나면 SageMaker 콘솔을 통해 SageMaker 노트북 인스턴스를 중지할 수 있습니다.

또한 지속적인 스토리지 비용을 방지하기 위해 생성한 S3 버킷/콘텐츠를 정리합니다.


In [ ]:
# TODO: Clean up any endpoints/etc to release resources